<a href="https://colab.research.google.com/github/Alexsaez1990/carto_exports_all_datasets/blob/main/Carto_export_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install carto

In [2]:
#AUTH CLIENT
import carto.auth
from carto.auth import APIKeyAuthClient

USERNAME="USERNAME"
USR_BASE_URL = f"https://{USERNAME}.carto.com/"
API_KEY = "YOUR_APIKEY"
auth_client = APIKeyAuthClient(api_key=API_KEY, base_url=USR_BASE_URL)
EXPORT_FORMAT = 'gpkg'
SAVE_FOLDER = '/content/drive/MyDrive/Carto_datasets'



In [ ]:
!pip install tqdm

In [ ]:
import argparse
import logging
import os
import warnings
from pathlib import Path

import carto.datasets
from carto.datasets import DatasetManager
from carto.auth import APIKeyAuthClient
from carto.sql import SQLClient

from tqdm import tqdm

warnings.filterwarnings('ignore')

# SQL wrapper
sql = SQLClient(APIKeyAuthClient(USR_BASE_URL, API_KEY))

# Dataset manager
dataset_manager = DatasetManager(auth_client)

# Get all datasets from account
datasets = dataset_manager.all()

# donwload datasets from account
for tablename in tqdm(datasets, desc="Processing..."):
    query = 'SELECT * FROM {table_name}'.format(table_name=tablename.name)
    data_folder = Path(SAVE_FOLDER) / "{table_name}.{format}".format(table_name=tablename.name, format=EXPORT_FORMAT)

    # Check if the file already exists in the save folder
    if not os.path.exists(data_folder):
        try:
            result = sql.send(query, format=EXPORT_FORMAT)
        except Exception as e:
            print.error(str(e))
            continue  # Skip to the next dataset

        # Write the file to the specified folder
        try:
            data_folder.write_bytes(result)
        except Exception as e:
            print(str(e))
    else:
        print(f"File for {tablename.name} already exists. Skipping download.")